In [2]:
import mysql.connector as con
import pandas as pd
import random
import datetime 
from datetime import datetime, timedelta


## ENUNCIADO EJERCICIO 

### 1. CREAR BASE DE DATOS (33%)

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.





In [3]:
def crear_tabla(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [4]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [5]:
crear_tabla(sql)

## Crear TABLAS

![alt text](image.png)

tiendas
- id_tienda (PRIMARY KEY)
- nombre_tienda
- direccion
- ciudad

In [6]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(255) NOT NULL, direccion VARCHAR(255), ciudad VARCHAR(255) NOT NULL);"

In [7]:
crear_tabla(sql)

empleados
- id_empleado (PRIMARY KEY)
- nombre_empleado
- puesto (ej.: Cajero, Gerente, Reponedor)
- id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [8]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(255) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [9]:
crear_tabla(sql)

categorias
- id_categoria (PRIMARY KEY)
- nombre_categoria

In [10]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [11]:
crear_tabla(sql)

productos
- id_producto (PRIMARY KEY)
- nombre_producto
- precio
- stock
- id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [12]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [13]:
crear_tabla(sql)

clientes
- id_cliente (PRIMARY KEY)
- first_name
- last_name
- email
- codigo_postal

In [14]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(200), codigo_postal INT);"

In [15]:
crear_tabla(sql)

ordenes
- id_orden (PRIMARY KEY)
- id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
- id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
- fecha_orden
- metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [16]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [17]:
crear_tabla(sql)

detalle_orden
- id_detalle (PRIMARY KEY)
- id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
- id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
- cantidad
- precio_unitario: mismo precio que en la tabla producto
- descuento (podría ser NULL si no se aplica)

In [18]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [19]:
crear_tabla(sql)

### 2. Generar datos demo desde Python (33%)

* Generar datos aleatorios en listas con Python similar los realizados en clase.
    * Uso de datetime, timedelta, random para generar datos aleatorios
* Pasar los datos a DataFrames de Pandas
* Pasar los DataFrames de Pandas a MySQL usando la función to_sql de Pandas con SQLAlchemy o usando MySQL Connector con sentencias INSERT.

In [20]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

tiendas

* id_tienda: valores enteros consecutivos (por ejemplo, 1, 2, 3...).
* nombre_tienda: nombres ficticios o genéricos (p. ej. "Tienda Centro", "Super Norte", "Super Sur").
* direccion: direcciones simples (p. ej. “Calle Falsa 123”).
* ciudad: usar nombres de ciudades ficticias o reales (p. ej. “Madrid”, “Barcelona”, “México DF”, etc.).

En total 5 o 10 tiendas.

In [21]:
tiendas = ['Jerez Este', 'Sanlucar Oeste', 'Cadiz Norte', 'Valdelagrana Sur', 'Arcos Sur', 'Chiclana Norte', 'Barbate Este', 'Vejer Oeste', 'Conil Sur', 'Rota Norte']
direcciones = ['Calle Catavino 25', 'Calle Manzanilla 42', 'Calle del Pan 38', 'Calle Playa 57', 'Calle Castillo 182', 'Calle Barrosa 63', 'Calle Atun 98', 'Calle Pescadill 56', 'Calle Balsa 71', 'Calle Base 22']
ciudades = ['Jerez de la Frontera', 'Sanlucar de Barrameda', 'Cadiz', 'El Puerto de Santa Maria', 'Arcos de la Frontera', 'Chiclana de la Frontera', 'Barbate', 'Vejer de la Frontera', 'Conil de la Frontera', 'Rota']
df_tiendas = pd.DataFrame(zip(tiendas, direcciones, ciudades), columns=['nombre_tienda', 'direccion', 'ciudad'])
# generar columna id_tienda
df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas

,id_tienda,nombre_tienda,direccion,ciudad
0,1,Jerez Este,Calle Catavino 25,Jerez de la Frontera
1,2,Sanlucar Oeste,Calle Manzanilla 42,Sanlucar de Barrameda
2,3,Cadiz Norte,Calle del Pan 38,Cadiz
3,4,Valdelagrana Sur,Calle Playa 57,El Puerto de Santa Maria
4,5,Arcos Sur,Calle Castillo 182,Arcos de la Frontera
5,6,Chiclana Norte,Calle Barrosa 63,Chiclana de la Frontera
6,7,Barbate Este,Calle Atun 98,Barbate
7,8,Vejer Oeste,Calle Pescadill 56,Vejer de la Frontera
8,9,Conil Sur,Calle Balsa 71,Conil de la Frontera
9,10,Rota Norte,Calle Base 22,Rota


In [22]:
insertar_dataframe(df_tiendas, 'tiendas')

10

empleados

* id_empleado: valores enteros consecutivos (1, 2, 3...).
* nombre_empleado: nombres y apellidos ficticios (p. ej. “Laura Gutiérrez”, “Juan Pérez”).
* puesto: limitarse a un conjunto reducido de valores (p. ej. {‘Cajero’, ‘Gerente’, ‘Reponedor’, ‘Vendedor’}).
* id_tienda: debe hacer referencia a una tienda existente (por ejemplo, un número entre 1 y 3 si solo tienes 3 tiendas).

En total 20 empleados por tienda.

In [23]:
#generar columna id_empleado
empleados = list()
nombres = ["Juan", "María", "Luis", "Ana", "Pedro", "Sofía", "Carlos", "Lucía", "Jorge", "Elena", "Manuel", "Laura", "Diego", "Clara", "Alberto", "Teresa", "Pablo", "Isabel", "Sergio", "Marta", "Roberto", "Julia", "Fernando", "Carmen", "Ricardo", "Paula", "Andrés", "Eva", "Antonio", "Sara", "José", "Natalia", "Francisco", "Patricia", "Hugo", "Adriana", "Rafael", "Irene", "Víctor", "Olga", "Daniel", "Noelia", "Álvaro", "Alicia", "Javier", "Rocío", "Enrique", "Amelia", "Gonzalo", "Vera"]
apellidos = ["García", "Martínez", "López", "Sánchez", "Pérez", "González", "Rodríguez", "Fernández", "Moreno", "Jiménez", "Hernández", "Muñoz", "Álvarez", "Romero", "Alonso", "Gutiérrez", "Ruiz", "Navarro", "Díaz", "Torres", "Castro", "Vázquez", "Ramos", "Gil", "Ramírez", "Flores", "Ortiz", "Marín", "Serrano", "Iglesias", "Delgado", "Domínguez", "Guerrero", "Molina", "Ortega", "Rubio", "Santos", "Medina", "Castillo", "Cruz", "Reyes", "Herrera", "Peña", "Carrasco", "Calvo", "Vega", "Cabrera", "Campos", "León", "Fuentes"]
trabajo = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Responsable de almacén", "Supervisor de caja", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de inventario", "Responsable de compras", "Promotor de ventas", "Encargado de seguridad"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = trabajo[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados

,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Francisco Romero,Cajero,1
1,2,Elena Gil,Reponedor,1
2,3,Andrés Castillo,Encargado de sección,1
3,4,Enrique Medina,Gerente,1
4,5,Luis Cruz,Personal de limpieza,1
...,...,...,...,...
195,196,Carlos Calvo,Auxiliar de caja,10
196,197,Francisco González,Encargado de inventario,10
197,198,Javier Martínez,Responsable de compras,10
198,199,Julia Ruiz,Promotor de ventas,10


In [24]:
insertar_dataframe(df_empleados, "empleados")

200

categorias

* id_categoria: valores enteros consecutivos (1, 2, 3...).
* nombre_categoria: selección de categorías (p. ej. “Lácteos”, “Carnes”, “Frutas”, “Verduras”, “Bebidas”, “Snacks”).

En total 10 categorías

In [25]:
categorias = ['Lácteos', 'Carnes', 'Frutas', 'Verduras', 'Bebidas', 'Snacks', 'Hogar', 'Limpieza', 'Perfumería', 'Jardín']

df_categorias = pd.DataFrame(categorias, columns=['nombre_categoria'])

# generar columna id_categoria
df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias

,id_categoria,nombre_categoria
0,1,Lácteos
1,2,Carnes
2,3,Frutas
3,4,Verduras
4,5,Bebidas
5,6,Snacks
6,7,Hogar
7,8,Limpieza
8,9,Perfumería
9,10,Jardín


In [26]:
insertar_dataframe(df_categorias, 'categorias')

10


productos

* id_producto: valores enteros consecutivos (1, 2, 3...).
* nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
* precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
* stock: valores INT entre 0 y 500 (o el rango que quieras).
* id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).

En total 4 productos de cada categoría.

In [27]:
# generar columna id_producto
produ = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(produ)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 100))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(produ, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,7.343864,23,1
1,2,Yogur,2.391993,6,1
2,3,Queso,7.968326,38,1
3,4,Mantequilla,5.916627,15,1
4,5,Pollo,5.237205,64,2
5,6,Carne de res,2.058330,85,2
6,7,Cerdo,9.204477,85,2
7,8,Chuletas,9.489109,61,2
8,9,Manzana,2.250036,11,3
9,10,Banana,7.091741,40,3


In [28]:
insertar_dataframe(df_productos, 'productos')

40

clientes

* id_cliente: valores enteros consecutivos (1, 2, 3...).
* nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
* email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
* telefono: número de 9 o 10 dígitos (dependiendo del país).
* direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).

En total: 2000.


In [29]:
# generar columna id_cliente
nombre_cliente, apellido_cliente, email, postal, cliente_completo = list(), list(), list(), list(), list()
calles = ("Av. Libertad", "Calle Mayor", "Paseo de la Reforma", "Calle Real", "Av. de la Constitución", "Calle del Sol", "Calle de la Luna", "Calle los Pinos", "Calle de la Paz", "Calle Olivo", "Calle Cedro", "Calle Nogal", "Av. de las Flores", "Paseo del Prado", "Calle del Río", "Av. Central", "Calle Primavera", "Calle Otoño", "Calle Invierno", "Calle Verano", "Calle Granada", "Calle Sevilla", "Calle Córdoba", "Calle Málaga", "Calle Valencia", "Calle Zaragoza", "Calle Alicante", "Av. de los Ángeles", "Calle del Carmen", "Calle San Miguel", "Calle San Juan", "Calle del Pilar", "Calle Santa Teresa", "Calle San Francisco", "Calle de la Sierra", "Calle del Mar", "Av. del Norte", "Av. del Sur", "Calle del Este", "Calle del Oeste", "Calle Diamante", "Calle Esmeralda", "Calle Rubí", "Calle Zafiro", "Calle Ámbar", "Calle Turquesa", "Calle Topacio", "Calle Perla", "Calle Horizonte", "Calle Amanecer")
for _ in range(2000):
    nom = random.choice(nombres)
    nombre_cliente.append(nom)
    ape = random.choice(apellidos)
    apellido_cliente.append(ape)
    email.append((f'{nom}.{ape}@suemail.com'))
    postal.append(random.randint(11000, 11900))
    
df_clientes = pd.DataFrame(zip(nombre_cliente, apellido_cliente, email, postal), columns=['first_name', 'last_name', 'email', 'codigo_postal']) 
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,first_name,last_name,email,codigo_postal
0,1,Diego,Gutiérrez,Diego.Gutiérrez@suemail.com,11098
1,2,Sergio,Fuentes,Sergio.Fuentes@suemail.com,11390
2,3,Sofía,Martínez,Sofía.Martínez@suemail.com,11878
3,4,Paula,Martínez,Paula.Martínez@suemail.com,11681
4,5,Patricia,Domínguez,Patricia.Domínguez@suemail.com,11345
...,...,...,...,...,...
1995,1996,Patricia,Ruiz,Patricia.Ruiz@suemail.com,11892
1996,1997,Sara,Vega,Sara.Vega@suemail.com,11307
1997,1998,Isabel,Castro,Isabel.Castro@suemail.com,11278
1998,1999,Marta,Fuentes,Marta.Fuentes@suemail.com,11619


In [30]:
insertar_dataframe(df_clientes, 'clientes')

2000

ordenes

* id_orden: valores enteros consecutivos (1, 2, 3...).
* id_cliente: haz referencia a los IDs existentes de la tabla clientes.
* id_empleado: haz referencia a los IDs existentes de la tabla empleados.
* fecha_orden: genera fechas aleatorias o secuenciales (p. ej. entre ‘2024-01-01’ y ‘2025-01-01’).
* metodo_pago: escoge entre {‘Tarjeta’, ‘Efectivo’}.

En total: 10000.


In [31]:
# generar columna id_orden
cliente, emplea, fecha, metodo = list(), list(), list(), list()
fechas = [datetime(year=2024, month=1, day=1) + timedelta(days=numero) for numero in range(395)]
for _ in range(10000):
    cliente.append(random.choice(df_clientes['id_cliente']))
    emplea.append(random.choice(df_empleados['id_empleado']))
    fecha.append(random.choice(fechas))
    metodo.append(random.choices(['Tarjeta', 'Efectivo'], weights=[0.65, 0.35])[0])

fecha.sort()
df_ordenes = pd.DataFrame(zip(cliente, emplea, fecha, metodo), columns=['id_cliente', 'id_empleado', 'fecha_orden', 'metodo_pago'])
df_ordenes.sort_values('fecha_orden', inplace=True)
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes.sort_values('id_orden', inplace=True)
df_ordenes.head(50)

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,1930,174,2024-01-01,Tarjeta
17,2,899,75,2024-01-01,Tarjeta
16,3,995,6,2024-01-01,Efectivo
15,4,449,55,2024-01-01,Tarjeta
14,5,656,46,2024-01-01,Tarjeta
13,6,1680,181,2024-01-01,Tarjeta
12,7,1511,48,2024-01-01,Efectivo
11,8,491,60,2024-01-01,Efectivo
10,9,1323,5,2024-01-01,Tarjeta
18,10,1275,21,2024-01-01,Tarjeta


In [32]:
insertar_dataframe(df_ordenes, 'ordenes')

10000

detalle_orden

* id_detalle: valores enteros consecutivos (1, 2, 3...).
* id_orden: referencia al ID de alguna orden válida.
* id_producto: referencia al ID de algún producto válido.
* cantidad: valores entre 1 y 20, por ejemplo.
* precio_unitario: usar el mismo precio que está en la tabla productos o uno ligeramente distinto si quieres simular ofertas.
* descuento: valores DECIMAL bajos (p. ej. 0.00, 1.00, 2.50) o NULL.

En total: 30000.

In [33]:
orden_completa = list()
productos = df_productos['id_producto']
orden_20 = df_ordenes['id_orden']
for orden in df_ordenes['id_orden']:
    ordenes = orden
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor, 2), round(descuento, 2)])   
for orden in range(20000):
    ordenes = random.choice(orden_20)
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor,2), round(descuento, 2)])   
df_detalle_orden = pd.DataFrame(orden_completa, columns=['id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento'])
df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,descuento
0,1,1,8,12,9.49,0.00
1,2,2,29,18,0.94,0.01
2,3,3,28,9,10.12,0.11
3,4,4,28,10,10.12,0.22
4,5,5,36,15,9.88,0.15
...,...,...,...,...,...,...
29995,29996,7462,17,6,6.91,0.18
29996,29997,1986,10,14,7.09,0.15
29997,29998,5519,8,3,9.49,0.09
29998,29999,1439,24,5,7.35,0.09


In [34]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

30000

### 3. Consultas SQL (34%)

In [35]:
def consultas(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
        return cursor.fetchall()
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

1. Listado de órdenes con detalles de cliente y empleado
* Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
* Utiliza un JOIN entre las tablas ordenes, clientes y empleados.

In [36]:
sql = ('SELECT o.id_orden, o.fecha_orden, c.first_name, c.last_name, e.nombre_empleado, o.metodo_pago FROM ordenes o JOIN clientes c ON c.id_cliente = o.id_cliente JOIN empleados e ON e.id_empleado = o.id_empleado ORDER BY o.id_orden;') 
consultas(sql)

[(1,
  datetime.date(2024, 1, 1),
  'Olga',
  'Vázquez',
  'Gonzalo Fernández',
  'Tarjeta'),
 (2,
  datetime.date(2024, 1, 1),
  'Ricardo',
  'Ramos',
  'Patricia Álvarez',
  'Tarjeta'),
 (3,
  datetime.date(2024, 1, 1),
  'Enrique',
  'Iglesias',
  'Sergio Iglesias',
  'Efectivo'),
 (4, datetime.date(2024, 1, 1), 'Julia', 'Romero', 'José Sánchez', 'Tarjeta'),
 (5,
  datetime.date(2024, 1, 1),
  'Ricardo',
  'Cabrera',
  'Amelia Vázquez',
  'Tarjeta'),
 (6, datetime.date(2024, 1, 1), 'Olga', 'Díaz', 'Pablo Ruiz', 'Tarjeta'),
 (7,
  datetime.date(2024, 1, 1),
  'Lucía',
  'Hernández',
  'Elena Moreno',
  'Efectivo'),
 (8, datetime.date(2024, 1, 1), 'Eva', 'Delgado', 'Luis Díaz', 'Efectivo'),
 (9, datetime.date(2024, 1, 1), 'Elena', 'Sánchez', 'Luis Cruz', 'Tarjeta'),
 (10,
  datetime.date(2024, 1, 1),
  'Roberto',
  'López',
  'Manuel Muñoz',
  'Tarjeta'),
 (11,
  datetime.date(2024, 1, 1),
  'Álvaro',
  'López',
  'Enrique Guerrero',
  'Tarjeta'),
 (12,
  datetime.date(2024, 1, 1),
  

2. Productos con stock bajo
* Filtra aquellos productos cuyo stock sea menor a 10.
* Muestra nombre del producto, categoría y stock.


In [37]:
sql = ('SELECT p.nombre_producto, c.nombre_categoria, p.stock FROM productos p JOIN categorias c ON p.id_categoria = c.id_categoria WHERE p.stock < 10;')
consultas(sql)

[('Yogur', 'Lácteos', 6), ('Papas fritas', 'Snacks', 4)]

3. Ventas totales por categoría
* Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
* Realiza el JOIN con detalle_orden, productos y categorias.
* Utiliza agrupación (GROUP BY).

In [38]:
sql = ('SELECT c.nombre_categoria as categoria, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as ventas_netas, sum(d.precio_unitario * d.cantidad) as ventas_brutas FROM categorias c JOIN productos p ON p.id_categoria = c.id_categoria JOIN detalle_orden d ON d.id_producto = p.id_producto group by c.id_categoria;')
consultas(sql)

[('Carnes', 201401.62, 201401.62),
 ('Limpieza', 153845.14, 153845.14),
 ('Hogar', 172332.85, 172332.85),
 ('Perfumería', 199882.86, 199882.86),
 ('Frutas', 182328.15, 182328.15),
 ('Snacks', 283837.2, 283837.2),
 ('Lácteos', 187404.56, 187404.56),
 ('Verduras', 193789.07, 193789.07),
 ('Bebidas', 185699.63, 185699.63),
 ('Jardín', 217066.01, 217066.01)]

4. Clientes con mayores gastos acumulados
* Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
* Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
* Ordena el resultado de mayor a menor gasto acumulado.

In [39]:
sql = ('SELECT c.first_name as nombre, c.last_name as apellido, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as compras FROM clientes c JOIN ordenes o ON c.id_cliente = o.id_cliente JOIN detalle_orden d ON d.id_orden = o.id_orden GROUP BY c.id_cliente ORDER BY sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) DESC;')
consultas(sql)

[('Vera', 'Jiménez', 4086.73),
 ('Eva', 'Medina', 3027.77),
 ('Olga', 'Serrano', 2978.51),
 ('Manuel', 'Reyes', 2895.15),
 ('Noelia', 'Serrano', 2888.27),
 ('Lucía', 'Marín', 2884.28),
 ('Paula', 'Vega', 2857.29),
 ('Lucía', 'Hernández', 2855.11),
 ('Isabel', 'García', 2854.47),
 ('Ricardo', 'Navarro', 2846.9),
 ('Álvaro', 'Navarro', 2815.32),
 ('Olga', 'Sánchez', 2799.26),
 ('Carmen', 'Ramos', 2792.96),
 ('Eva', 'González', 2634.79),
 ('Sara', 'Castro', 2634.49),
 ('Ana', 'Peña', 2625.91),
 ('Isabel', 'Peña', 2571.6),
 ('Alberto', 'Moreno', 2563.45),
 ('Pablo', 'Reyes', 2546.23),
 ('Alicia', 'Torres', 2541.52),
 ('Manuel', 'Molina', 2538.45),
 ('Alicia', 'Serrano', 2490.47),
 ('Rafael', 'Ortiz', 2464.6),
 ('Natalia', 'Pérez', 2462.25),
 ('Julia', 'Sánchez', 2417.92),
 ('Antonio', 'Ortiz', 2399.94),
 ('Clara', 'Domínguez', 2394.12),
 ('Álvaro', 'Ruiz', 2384.78),
 ('Manuel', 'Fuentes', 2382.64),
 ('Olga', 'Alonso', 2376.51),
 ('Fernando', 'López', 2360.36),
 ('Francisco', 'Calvo', 2355.

5. Empleados y número de órdenes gestionadas
* Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
* Utiliza GROUP BY y COUNT.

In [40]:
sql = ('SELECT e.nombre_empleado AS EMPLEADO, e.puesto AS PUESTO, count(o.id_empleado) AS ORDENES FROM empleados e JOIN ordenes o ON o.id_empleado = e.id_empleado GROUP BY e.id_empleado')
consultas(sql)

[('Francisco Romero', 'Cajero', 53),
 ('Elena Gil', 'Reponedor', 47),
 ('Andrés Castillo', 'Encargado de sección', 46),
 ('Enrique Medina', 'Gerente', 53),
 ('Luis Cruz', 'Personal de limpieza', 51),
 ('Sergio Iglesias', 'Panadero', 49),
 ('Daniel Hernández', 'Carnicero', 52),
 ('Rocío Vega', 'Pescadero', 59),
 ('Eva Campos', 'Responsable de almacén', 47),
 ('Marta Cruz', 'Supervisor de caja', 43),
 ('Julia Fernández', 'Frutero', 49),
 ('Francisco Hernández', 'Charcutero', 47),
 ('Isabel García', 'Jefe de turno', 39),
 ('Gonzalo Medina', 'Operador logístico', 49),
 ('Noelia Ortega', 'Atención al cliente', 43),
 ('José Serrano', 'Auxiliar de caja', 39),
 ('Daniel Jiménez', 'Encargado de inventario', 57),
 ('Irene Herrera', 'Responsable de compras', 55),
 ('Pablo Ramírez', 'Promotor de ventas', 55),
 ('Sofía Ruiz', 'Encargado de seguridad', 54),
 ('Manuel Muñoz', 'Cajero', 60),
 ('Olga Vázquez', 'Reponedor', 52),
 ('Clara González', 'Encargado de sección', 57),
 ('Andrés Domínguez', 'Ger

6. Ordenes filtradas por fecha y tienda
* Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
* Incluye datos de la tienda y del cliente.

In [41]:
sql = ("SELECT t.id_tienda, t.nombre_tienda AS tienda, o.id_orden AS factura, o.fecha_orden AS fecha, concat(c.first_name,' ',c.last_name) AS cliente FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON o.id_empleado = e.id_empleado JOIN clientes c ON c.id_cliente = o.id_cliente WHERE T.id_tienda = 1 and o.fecha_orden > '2024-01-31' and o.fecha_orden < '2024-03-01' ORDER BY o.fecha_orden;")
consultas(sql)

[(1, 'Jerez Este', 741, datetime.date(2024, 2, 1), 'Isabel Ortega'),
 (1, 'Jerez Este', 752, datetime.date(2024, 2, 1), 'Andrés Rodríguez'),
 (1, 'Jerez Este', 768, datetime.date(2024, 2, 2), 'Pablo Pérez'),
 (1, 'Jerez Este', 783, datetime.date(2024, 2, 2), 'Alicia Ruiz'),
 (1, 'Jerez Este', 797, datetime.date(2024, 2, 3), 'Antonio Torres'),
 (1, 'Jerez Este', 807, datetime.date(2024, 2, 4), 'Alberto Iglesias'),
 (1, 'Jerez Este', 830, datetime.date(2024, 2, 4), 'Paula Ortiz'),
 (1, 'Jerez Este', 845, datetime.date(2024, 2, 5), 'Fernando Alonso'),
 (1, 'Jerez Este', 860, datetime.date(2024, 2, 6), 'María Santos'),
 (1, 'Jerez Este', 865, datetime.date(2024, 2, 6), 'Francisco Herrera'),
 (1, 'Jerez Este', 878, datetime.date(2024, 2, 6), 'Rafael Sánchez'),
 (1, 'Jerez Este', 884, datetime.date(2024, 2, 6), 'Daniel Ortega'),
 (1, 'Jerez Este', 899, datetime.date(2024, 2, 7), 'Rocío Vega'),
 (1, 'Jerez Este', 951, datetime.date(2024, 2, 10), 'Natalia Martínez'),
 (1, 'Jerez Este', 963, da

7. Ranking de productos más vendidos en cada tienda
* Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
* Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
* Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).


In [ ]:
#CON MUCHA AYUDA!!!!
sql = ('WITH ranking_productos AS (SELECT t.nombre_tienda, p.nombre_producto, SUM(do.cantidad) AS total_vendido, ROW_NUMBER() OVER (PARTITION BY t.id_tienda ORDER BY SUM(do.cantidad) DESC) AS ranking FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON e.id_empleado = o.id_empleado JOIN detalle_orden do ON o.id_orden = do.id_orden JOIN productos p ON do.id_producto = p.id_producto GROUP BY t.id_tienda, t.nombre_tienda, p.id_producto, p.nombre_producto) SELECT nombre_tienda, nombre_producto, total_vendido FROM ranking_productos WHERE ranking <= 3 ORDER BY nombre_tienda, ranking;')
consultas(sql)

[('Arcos Sur', 'Pollo', 1005.0),
 ('Arcos Sur', 'Leche', 993.0),
 ('Arcos Sur', 'Jugo de naranja', 935.0),
 ('Barbate Este', 'Crema hidratante', 1038.0),
 ('Barbate Este', 'Desinfectante', 1032.0),
 ('Barbate Este', 'Detergente', 1023.0),
 ('Cadiz Norte', 'Chocolate', 1042.0),
 ('Cadiz Norte', 'Pilas', 935.0),
 ('Cadiz Norte', 'Refresco', 926.0),
 ('Chiclana Norte', 'Chicles', 1048.0),
 ('Chiclana Norte', 'Cebolla', 1040.0),
 ('Chiclana Norte', 'Champú', 1029.0),
 ('Conil Sur', 'Champú', 937.0),
 ('Conil Sur', 'Cebolla', 918.0),
 ('Conil Sur', 'Refresco', 902.0),
 ('Jerez Este', 'Detergente', 1048.0),
 ('Jerez Este', 'Jabón líquido', 1024.0),
 ('Jerez Este', 'Tomate', 971.0),
 ('Rota Norte', 'Chocolate', 1028.0),
 ('Rota Norte', 'Galletas', 1015.0),
 ('Rota Norte', 'Esponjas', 1003.0),
 ('Sanlucar Oeste', 'Mantequilla', 1013.0),
 ('Sanlucar Oeste', 'Zanahoria', 933.0),
 ('Sanlucar Oeste', 'Crema hidratante', 932.0),
 ('Valdelagrana Sur', 'Uva', 1107.0),
 ('Valdelagrana Sur', 'Pollo', 9

Opcional: añadir alguna consulta con subconsultas o algo que no se abarque en las anteriores consulta.

In [43]:
sql = ('')
consultas(sql)

Ha ocurrido un error: No result set to fetch from


0